### Этап 1. Разворачивание MLflow Tracking Server и MLflow Model Registry. Регистрация существующей модели

In [21]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import CatBoostEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import KFold, cross_validate
from catboost import CatBoostRegressor

import mlflow
import joblib

In [2]:
load_dotenv()

True

In [4]:
host = os.environ.get('DB_DESTINATION_HOST')
port = os.environ.get('DB_DESTINATION_PORT')
username = os.environ.get('DB_DESTINATION_USER')
password = os.environ.get('DB_DESTINATION_PASSWORD')
db = os.environ.get('DB_DESTINATION_NAME')

In [13]:
conn = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db}', connect_args={'sslmode':'require'})
data = pd.read_sql('select * from clean_flats_dataset', conn, index_col='flat_id')

In [14]:
# Вместо года постройки добавляем возраст здания
data['building_age'] = (datetime.now().year - data['build_year']).astype('float')

# Удаляем лишние колонки
data.drop(
    columns=['id', 'build_year', 'studio'], 
    inplace=True
)

# Изменяем тип building_type_int на object
data['building_type_int'] = data['building_type_int'].astype('object')

# Изменяем тип количественных целых признаков на float
num_int_cols = data.select_dtypes('int').columns
data[num_int_cols] = data[num_int_cols].astype('float') 

In [19]:
data.head()

,floor,kitchen_area,living_area,rooms,is_apartment,total_area,price,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,building_age
flat_id,,,,,,,,,,,,,,,
8348,8.0,10.6,56.000000,3.0,False,88.599998,10990000.0,4,55.542187,37.483067,2.64,409.0,18.0,True,6.0
8350,3.0,7.0,28.000000,2.0,False,44.700001,8999000.0,4,55.857765,37.422684,2.64,143.0,9.0,True,57.0
8351,16.0,10.9,54.799999,4.0,False,89.099998,24000000.0,4,55.562908,37.570431,2.70,164.0,16.0,True,28.0
8352,2.0,7.4,66.300003,4.0,False,93.000000,17500000.0,1,55.653507,37.649426,2.70,59.0,6.0,True,59.0
8354,4.0,9.1,17.700001,1.0,False,34.000000,7500000.0,1,55.796406,37.459873,3.00,72.0,9.0,True,60.0


In [32]:
os.makedirs('../data', exist_ok=True)
data.to_csv('../data/initial_data.csv', index_label='flat_id')

In [23]:
cat_features = data.select_dtypes(include=['bool', 'object'])
is_binary_cat_features = cat_features.nunique() == 2
binary_cat_features = cat_features[is_binary_cat_features[is_binary_cat_features].index]
other_cat_features = cat_features[is_binary_cat_features[~is_binary_cat_features].index]
num_features = data.select_dtypes(['float']) 

preprocessor = ColumnTransformer(
    [
        ('binary_cat', OneHotEncoder(drop='if_binary'), binary_cat_features.columns.tolist()),
        ('other_cat', CatBoostEncoder(), other_cat_features.columns.tolist()),
        ('num', StandardScaler(), num_features.columns.tolist())
    ],
    remainder='drop',
    verbose_feature_names_out=False
)

In [22]:
model = CatBoostRegressor(loss_function='MAPE', random_state=42)
    
pipeline = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('model', model)
    ]
)

In [26]:
pipeline.fit(data, data['price'])

0:	learn: 0.9718547	total: 86ms	remaining: 1m 25s
1:	learn: 0.9446030	total: 108ms	remaining: 54.1s
2:	learn: 0.9181978	total: 138ms	remaining: 46s
3:	learn: 0.8926607	total: 160ms	remaining: 39.9s
4:	learn: 0.8681229	total: 181ms	remaining: 36.1s
5:	learn: 0.8442053	total: 203ms	remaining: 33.6s
6:	learn: 0.8219271	total: 224ms	remaining: 31.7s
7:	learn: 0.7994399	total: 245ms	remaining: 30.3s
8:	learn: 0.7781502	total: 264ms	remaining: 29.1s
9:	learn: 0.7572879	total: 286ms	remaining: 28.3s
10:	learn: 0.7371173	total: 308ms	remaining: 27.7s
11:	learn: 0.7165772	total: 328ms	remaining: 27s
12:	learn: 0.6985965	total: 349ms	remaining: 26.5s
13:	learn: 0.6809005	total: 370ms	remaining: 26s
14:	learn: 0.6629743	total: 391ms	remaining: 25.6s
15:	learn: 0.6464993	total: 413ms	remaining: 25.4s
16:	learn: 0.6315422	total: 446ms	remaining: 25.8s
17:	learn: 0.6169486	total: 471ms	remaining: 25.7s
18:	learn: 0.6029749	total: 495ms	remaining: 25.5s
19:	learn: 0.5892968	total: 517ms	remaining: 25

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('binary_cat',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['is_apartment',
                                                   'has_elevator']),
                                                 ('other_cat',
                                                  CatBoostEncoder(),
                                                  ['building_type_int']),
                                                 ('num', StandardScaler(),
                                                  ['floor', 'kitchen_area',
                                                   'living_area', 'rooms',
                                                   'total_area', 'price',
                                                   'latitude', 'longitude',
                                                   'ceiling_height',
                                                   'flats_count',
                                                   'floors_total',
                                                   'building_age'])],
                                   verbose_feature_names_out=False)),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x7f3cad3e2f50>)])

In [27]:
os.makedirs('../models', exist_ok=True)
with open('../models/flats_price_baseline_model.pkl', 'wb') as fd:
    joblib.dump(pipeline, fd) 

In [24]:
cv_strategy = KFold(
    n_splits=5, 
    shuffle=True,
    random_state=42
)

cv_res = cross_validate(
    pipeline,
    data,
    data['price'],
    cv=cv_strategy,
    n_jobs=-1,
    scoring='neg_mean_absolute_percentage_error'
)

for key, value in cv_res.items():
    cv_res[key] = round(value.mean(), 3)

0:	learn: 0.9717890	total: 162ms	remaining: 2m 41s
0:	learn: 0.9719470	total: 162ms	remaining: 2m 42s
1:	learn: 0.9447070	total: 206ms	remaining: 1m 42s
1:	learn: 0.9447779	total: 210ms	remaining: 1m 44s
2:	learn: 0.9180343	total: 250ms	remaining: 1m 23s
2:	learn: 0.9185475	total: 251ms	remaining: 1m 23s
3:	learn: 0.8927514	total: 291ms	remaining: 1m 12s
3:	learn: 0.8929024	total: 303ms	remaining: 1m 15s
4:	learn: 0.8673588	total: 331ms	remaining: 1m 5s
4:	learn: 0.8672941	total: 347ms	remaining: 1m 8s
5:	learn: 0.8435169	total: 371ms	remaining: 1m 1s
5:	learn: 0.8433228	total: 394ms	remaining: 1m 5s
6:	learn: 0.8203825	total: 419ms	remaining: 59.4s
6:	learn: 0.8204048	total: 436ms	remaining: 1m 1s
7:	learn: 0.7980378	total: 467ms	remaining: 57.9s
7:	learn: 0.7971924	total: 490ms	remaining: 1m
8:	learn: 0.7763543	total: 525ms	remaining: 57.9s
8:	learn: 0.7744281	total: 530ms	remaining: 58.4s
9:	learn: 0.7559765	total: 568ms	remaining: 56.3s
9:	learn: 0.7535766	total: 574ms	remaining: 5

In [28]:
cv_res

{'fit_time': 40.533, 'score_time': 0.267, 'test_score': -0.407}

In [29]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" 
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") 
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") 

TRACKING_SERVER_HOST = '127.0.0.1' 
TRACKING_SERVER_PORT = 5000

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 

EXPERIMENT_NAME = 'mle-project-sprint-2'
RUN_NAME = 'flats_price_baseline_model_logging'
REGISTRY_MODEL_NAME = 'flats_price_baseline_model'

In [33]:
class CustomMlflowModel(mlflow.pyfunc.PythonModel):
    def __init__(self, model):
        super().__init__()
        self._model = model
    
    def predict(self, context, model_input):
        return self._model.predict(model_input)

In [34]:
custom_model = CustomMlflowModel(pipeline)

In [44]:
metadata = {'model_type': 'regression'}
pip_requirements = "../requirements.txt" 
signature = mlflow.models.infer_signature(
    data,
    pipeline.predict(data)
) 
input_example = data[:10]
params = {
    'tree_count': model.tree_count_,
    'learning_rate': model.learning_rate_
}

#experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_metrics(cv_res)
    mlflow.log_params(params)
    mlflow.log_artifact("../data/initial_data.csv", "artifacts")
    model_info = mlflow.pyfunc.log_model( 
        python_model=custom_model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        metadata=metadata,
        signature=signature,
        input_example=input_example,
        pip_requirements=pip_requirements
    )

/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Successfully registered model 'flats_price_baseline_model'.
2024/04/14 20:58:27 INFO mlflow.tracking._model_registry.client: Waiting up to 3